# Convolutional Neural Network

### Importing the libraries

In [15]:
!pip install --upgrade tensorflow keras Pillow

You should consider upgrading via the 'C:\Users\Chaydson\anaconda3\python.exe -m pip install --upgrade pip' command.


In [16]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [17]:
tf.__version__

'2.8.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [18]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [19]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [20]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [21]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [22]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [23]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [24]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [25]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [26]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [27]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [28]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 48s 190ms/step - loss: 0.6890 - accuracy: 0.5257 - val_loss: 0.6779 - val_accuracy: 0.5725
Epoch 2/25
250/250 [==============================] - 33s 130ms/step - loss: 0.6523 - accuracy: 0.6101 - val_loss: 0.6135 - val_accuracy: 0.6740
Epoch 3/25
250/250 [==============================] - 32s 130ms/step - loss: 0.6070 - accuracy: 0.6681 - val_loss: 0.5819 - val_accuracy: 0.7070
Epoch 4/25
250/250 [==============================] - 34s 137ms/step - loss: 0.5581 - accuracy: 0.7179 - val_loss: 0.5361 - val_accuracy: 0.7430
Epoch 5/25
250/250 [==============================] - 35s 139ms/step - loss: 0.5291 - accuracy: 0.7347 - val_loss: 0.5132 - val_accuracy: 0.7480
Epoch 6/25
250/250 [==============================] - 34s 136ms/step - loss: 0.5050 - accuracy: 0.7470 - val_loss: 0.4727 - val_accuracy: 0.7765
Epoch 7/25
250/250 [==============================] - 34s 136ms/step - loss: 0.4898 - accuracy: 0.7639 - val_loss: 0.4762 - val_ac

## Part 4 - Making a single prediction

In [29]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [30]:
print(prediction)

dog


## Fim